In [1]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization , MaxPooling2D,GlobalAveragePooling2D,UpSampling2D , concatenate, multiply ,Concatenate,AveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.metrics import AUC, BinaryAccuracy, Precision, Recall, FalseNegatives, FalsePositives, TrueNegatives, TruePositives
from tensorflow.keras.losses import BinaryCrossentropy
import os
import math
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, BatchNormalization, Input, Lambda, Multiply
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Lambda, Dense, Reshape
from keras.regularizers import l2


2023-07-04 10:38:23.649617: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14508316849998821480
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5750063104
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5085153504745099307
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060 Ti, pci bus id: 0000:2b:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


2023-07-03 21:23:37.270105: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-03 21:23:37.284556: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-03 21:23:37.284700: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-07-03 21:23:38.828453: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-03 21:23:38.828609: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-03 21:23:38.828711: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [4]:
# Set the GPU device
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    tf.config.experimental.set_visible_devices(physical_devices[0], 'GPU')

In [5]:
# Set GPU device
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # Replace '0' with the GPU device ID

In [6]:
np.random.seed(42)
tf.random.set_seed(42)


In [7]:
IMG_HEIGHT = 299
IMG_WIDTH = 299
BATCH_SIZE = 8


In [8]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)


In [9]:
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)

In [10]:
train_path = "/media/maged/New Volume/NU/1-Spring 23/Grad Project 2/datasets again/Defacto + others/train"
test_path = "/media/maged/New Volume/NU/1-Spring 23/Grad Project 2/datasets again/Defacto + others/test"
valid_path = "/media/maged/New Volume/NU/1-Spring 23/Grad Project 2/datasets again/Defacto + others/validation"

In [11]:
train_batches = train_datagen.flow_from_directory(
    train_path,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=True
)

valid_batches = valid_datagen.flow_from_directory(
    valid_path,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=True
)

Found 33565 images belonging to 2 classes.
Found 5753 images belonging to 2 classes.


In [12]:
train_image_count = train_batches.samples
valid_image_count = valid_batches.samples

In [13]:
steps_per_epoch = math.ceil(train_image_count / BATCH_SIZE)
validation_steps = math.ceil(valid_image_count / BATCH_SIZE)


In [14]:
base_model = InceptionV3(weights= 'imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))


2023-07-03 21:23:43.990481: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-03 21:23:43.990681: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-03 21:23:43.990807: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [15]:
for layer in base_model.layers:
    layer.trainable = False


In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu', kernel_regularizer=l2(0.001))(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu', kernel_regularizer=l2(0.001))(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

In [15]:
# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

In [17]:
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)

In [23]:
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)

In [21]:
x = base_model.output
attention = Conv2D(1, (1, 1), activation='sigmoid')(x)
attended_features = multiply([x, attention])   
    
# Classification layers
x = Flatten()(attended_features)
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

# Output layer
output = Dense(1, activation='sigmoid')(x)

In [27]:
# Add new classification layers on top
x1 = base_model.output
x1 = GlobalAveragePooling2D()(x1)
x1 = Dense(256, activation='relu')(x1)

# Add local feature branch
x2 = Conv2D(128, (3, 3), activation='relu')(base_model.output)
x2 = MaxPooling2D(pool_size=(2, 2))(x2)
x2 = Flatten()(x2)
x2 = Dense(256, activation='relu')(x2)

# Concatenate global and local features
x = concatenate([x1, x2])
output = Dense(1, activation='sigmoid')(x)

In [10]:
# Add new classification layers on top
x1 = base_model.output
x1 = GlobalAveragePooling2D()(x1)
x1 = Dense(512, activation='relu')(x1)
x1 = BatchNormalization()(x1)
x1 = Dropout(0.5)(x1)
x1 = Dense(256, activation='relu')(x1)
x1 = BatchNormalization()(x1)
x1 = Dropout(0.5)(x1)

# Add local feature branch
x2 = Conv2D(256, (3, 3), activation='relu')(base_model.output)
x2 = MaxPooling2D(pool_size=(2, 2))(x2)
x2 = Flatten()(x2)
x2 = Dense(512, activation='relu')(x2)
x2 = BatchNormalization()(x2)
x2 = Dropout(0.5)(x2)
x2 = Dense(256, activation='relu')(x2)
x2 = BatchNormalization()(x2)
x2 = Dropout(0.5)(x2)

# Reshape x1 to match dimensions for attention
x1_reshape = Reshape((1, 1, 256))(x1)

# Spatial Attention
att1 = Conv2D(1, (1, 1), activation='sigmoid')(x1_reshape)
x1_att = Multiply()([x1, att1])

# Flatten x1_att
x1_att_flat = Flatten()(x1_att)

# Channel-wise Attention
att2 = Dense(256, activation='sigmoid')(x1_att_flat)
att2 = Reshape((1, 1, 256))(att2)
x2_att = Multiply()([x2, att2])
x2_att = Flatten()(x2_att)

# Concatenate global and local features
x = concatenate([x1_att_flat, x2_att])
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)


In [10]:
# Input shape and number of classes
input_shape = (224, 224, 3)
num_classes = 2  # Two classes: original and manipulated

# Input layer
input_layer = Input(shape=input_shape)

# Convolutional layers
conv1 = Conv2D(32, (3, 3), strides=(2, 2), activation='relu')(input_layer)
conv2 = Conv2D(64, (3, 3), activation='relu')(conv1)
conv2_pool = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(conv2)
conv3 = Conv2D(128, (3, 3), padding='valid', activation='relu')(conv2_pool)
# ...

# Inception Module (Mixed 0)
conv1x1_0 = Conv2D(64, (1, 1), activation='relu')(conv3)

conv3x3_0 = Conv2D(96, (1, 1), activation='relu')(conv3)
conv3x3_0 = Conv2D(128, (3, 3), padding='same', activation='relu')(conv3x3_0)

conv5x5_0 = Conv2D(16, (1, 1), activation='relu')(conv3)
conv5x5_0 = Conv2D(32, (5, 5), padding='same', activation='relu')(conv5x5_0)

pool_0 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(conv3)
pool_0 = Conv2D(32, (1, 1), activation='relu')(pool_0)

mixed0 = Concatenate(axis=-1)([conv1x1_0, conv3x3_0, conv5x5_0, pool_0])

# Inception Module (Mixed 1)
conv1x1_1 = Conv2D(128, (1, 1), actregulritazationivation='relu')(mixed0)

conv3x3_1 = Conv2D(128, (1, 1), activation='relu')(mixed0)
conv3x3_1 = Conv2D(192, (3, 3), padding='same', activation='relu')(conv3x3_1)

conv5x5_1 = Conv2D(32, (1, 1), activation='relu')(mixed0)
conv5x5_1 = Conv2D(96, (5, 5), padding='same', activation='relu')(conv5x5_1)

pool_1 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(mixed0)
pool_1 = Conv2D(64, (1, 1), activation='relu')(pool_1)

mixed1 = Concatenate(axis=-1)([conv1x1_1, conv3x3_1, conv5x5_1, pool_1])

# ...

# Auxiliary Classifier
aux_pooling = AveragePooling2D(pool_size=(5, 5), strides=(3, 3))(mixed1)  # Replace mixedX with the appropriate layer
aux_conv = Conv2D(128, (1, 1), activation='relu')(aux_pooling)
aux_flat = Flatten()(aux_conv)
aux_fc1 = Dense(1024, activation='relu')(aux_flat)
aux_dropout = Dropout(0.7)(aux_fc1)
aux_output = Dense(num_classes, activation='softmax')(aux_dropout)

# Fully Connected Layers
flatten = Flatten()(mixed1)  # Replace mixedX with the appropriate layer
fc1 = Dense(2048, activation='relu')(flatten)
dropout = Dropout(0.5)(fc1)
output = Dense(num_classes, activation='softmax')(dropout)

2023-07-02 20:44:52.156055: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 10.29GiB (rounded to 11045437440)requested by op StatelessRandomUniformV2
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-07-02 20:44:52.156124: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2023-07-02 20:44:52.156134: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 161, Chunks in use: 161. 40.2KiB allocated for chunks. 40.2KiB in use in bin. 14.3KiB client-requested in use in bin.
2023-07-02 20:44:52.156139: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 174, Chunks in use: 174. 117.8KiB allocated for chunks. 117.8KiB in use in bin. 109.3KiB client-requested in use in bin.
2023-07-02 20:44:52.156143:

ResourceExhaustedError: {{function_node __wrapped__StatelessRandomUniformV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[1348320,2048] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:StatelessRandomUniformV2]

In [26]:
# Add custom layers for copy-move forgery detection
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x) 
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x) 
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x) 
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x) 

In [22]:
model = Model(inputs=base_model.input, outputs=predictions)


In [23]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss=BinaryCrossentropy(),
              metrics=[AUC(), BinaryAccuracy(), Precision(), Recall(), FalseNegatives(), FalsePositives(), TrueNegatives(), TruePositives()])

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 149, 149, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                           

In [18]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.0000001, verbose=1)


In [23]:
checkpoint_path = '/media/maged/New Volume/NU/1-Spring 23/Grad Project 2/Attempt/inception V3 copy move.h5'

In [24]:
model.load_weights(checkpoint_path)

In [19]:
history = model.fit(
    train_batches,
    verbose=1,
    validation_data=valid_batches,
    epochs=50,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=[early_stopping,reduce_lr]
)

Epoch 1/50


2023-07-03 21:24:00.036776: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-07-03 21:24:07.528025: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-07-03 21:24:10.571984: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-07-03 21:24:10.576166: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x32ce3350 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-07-03 21:24:10.576182: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Ti, Compute Capability 8.6
2023-07-03 21:24:10.579601

4196/4196 [==============================] - ETA: 0s - loss: 0.6963 - auc: 0.5096 - binary_accuracy: 0.5118 - precision: 0.5050 - recall: 0.2282 - false_negatives: 12706.0000 - false_positives: 3682.0000 - true_negatives: 13421.0000 - true_positives: 3756.0000

2023-07-03 21:47:01.265500: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


4196/4196 [==============================] - 1545s 361ms/step - loss: 0.6963 - auc: 0.5096 - binary_accuracy: 0.5118 - precision: 0.5050 - recall: 0.2282 - false_negatives: 12706.0000 - false_positives: 3682.0000 - true_negatives: 13421.0000 - true_positives: 3756.0000 - val_loss: 1.0324 - val_auc: 0.5322 - val_binary_accuracy: 0.5084 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_false_negatives: 2828.0000 - val_false_positives: 0.0000e+00 - val_true_negatives: 2925.0000 - val_true_positives: 0.0000e+00 - lr: 0.0010
Epoch 2/50
4196/4196 [==============================] - 1229s 293ms/step - loss: 0.6930 - auc: 0.5092 - binary_accuracy: 0.5099 - precision: 0.5013 - recall: 0.1287 - false_negatives: 14344.0000 - false_positives: 2107.0000 - true_negatives: 14996.0000 - true_positives: 2118.0000 - val_loss: 0.6929 - val_auc: 0.5032 - val_binary_accuracy: 0.5084 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_false_negatives: 2828.0000 - val_false_positives: 0.0000e+

In [20]:
model.save('inception V3 copy move_GPT7.h5')

In [39]:
model.save('Final Model.json')

INFO:tensorflow:Assets written to: Final Model.json\assets


INFO:tensorflow:Assets written to: Final Model.json\assets


In [36]:
model.save_weights('D:\\Uni\\Grad\\2\\Attempt\\My Final Model Weights.h5')

In [42]:
Customow.keras.models import model_from_json
# load json and create model
json_file = open('Final Model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

PermissionError: [Errno 13] Permission denied: 'Final Model.json\\assets'

In [43]:
# Define your data generator
class CustomImageDataGenerator(tf.keras.preprocessing.image.ImageDataGenerator):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def flow_from_directory(self, *args, **kwargs):
        iterator = super().flow_from_directory(*args, **kwargs)
        while True:
            try:
                data = next(iterator)
                yield data
            except (OSError, tf.errors.InvalidArgumentError):
                continue

In [38]:
model = tf.keras.models.load_model('/media/maged/New Volume/NU/1-Spring 23/Grad Project 2/Attempt/inception V3 copy move_4.h5')


In [39]:
img_data_gn = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [40]:
test_batches = img_data_gn.flow_from_directory(test_path,
                                               target_size=(IMG_HEIGHT, IMG_WIDTH),
                                               class_mode='binary',
                                               classes=['authentic', 'tampered'],
                                               batch_size=8,
                                               shuffle=False)

Found 4446 images belonging to 2 classes.


In [41]:
test_image_count = sum([len(files) for _, _, files in os.walk(test_path)])
test_steps = math.ceil(test_image_count / BATCH_SIZE)

In [42]:
model.compile(Adam(learning_rate=0.001), loss=BinaryCrossentropy(),
                 metrics=[AUC(), BinaryAccuracy(), Precision(), Recall(), FalseNegatives(),
                          FalsePositives(), TrueNegatives(), TruePositives() ])

In [36]:
test_loss, test_accuracy = model.evaluate(test_batches, steps=test_steps)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

2023-07-03 09:37:11.688444: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


556/556 [==============================] - 20s 32ms/step - loss: 0.7643 - auc_3: 0.7127 - binary_accuracy: 0.6392 - precision_3: 0.6057 - recall_3: 0.7754 - false_negatives_3: 494.0000 - false_positives_3: 1110.0000 - true_negatives_3: 1137.0000 - true_positives_3: 1705.0000


ValueError: too many values to unpack (expected 2)

In [43]:
# Evaluate the model using the test generator
predictions = model.predict(test_batches)

# Get the predicted labels
predicted_labels = np.argmax(predictions, axis=1)

# Get the true labels from the generator
true_labels = test_batches.classes

# Calculate accuracy
accuracy = np.mean(predicted_labels == true_labels)
print(f"Accuracy: {accuracy}")

2023-07-03 09:41:09.843105: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


556/556 [==============================] - 20s 33ms/step
Accuracy: 0.5053981106612686


In [28]:
import os

test_directory = "/media/maged/New Volume/NU/1-Spring 23/Grad Project 2/datasets again/Defacto + others/test" # Replace with the path to your test directory

test_labels = []

authentic_folder = os.path.join(test_directory, "authentic")
tampered_folder = os.path.join(test_directory, "tampered")

authentic_files = os.listdir(authentic_folder)
tampered_files = os.listdir(tampered_folder)

for _ in range(len(authentic_files)):
    test_labels.append(0)  # 0 represents the label for authentic images

for _ in range(len(tampered_files)):
    test_labels.append(1)  # 1 represents the label for tampered images

print(test_labels)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [29]:
y_pred = model.predict(test_batches)
y_pred_classes = np.argmax(y_pred, axis=1)
accuracy = np.mean(y_pred_classes == test_labels)
print(f"Accuracy: {accuracy}")


2023-07-03 09:33:21.784055: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


19/19 [==============================] - 19s 982ms/step
Accuracy: 0.5053981106612686


In [ ]:
import h5py

with h5py.File('/home/maged/ResNet50_3.h5', 'r') as f:
    print(list(f.keys()))  # Print the keys of groups and datasets


In [ ]:
model_json = newModel.to_json()


In [ ]:
with open("CNNImageSplicingDetectorModelResNet50_4k.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
for steps in range(len(test_batches)):
    test_images, test_labels = next(test_batches)
    (newModel.evaluate(test_images, test_labels))


In [ ]:
#serialize weights to HDF5
newModel.save_weights("ResNet50.h5")


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
um_test_samples = len(test_batches.labels)

# Generate predictions for the test data using the model
y_pred = model.predict(test_batches)
y_pred_classes = np.argmax(y_pred, axis=1)

# Get the true labels from the test data generator
y_true = test_batches.labels

# Generate the confusion matrix
cm = confusion_matrix(y_true, y_pred_classes)

# Plot the confusion matrix
class_labels = test_batches.class_indices.keys()
plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, cmap='Blues', fmt='d',
            xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()